## First import necessary packages

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import imdb
ia = imdb.Cinemagoer()

## Import the title.basics csv file that contains genre, runtime, and movie title information

In [2]:
imdb_data = pd.read_csv("title.basics.tsv", sep = "\t")

/var/folders/ks/69t3xv4s3t35zdzb9tftzcqc0000gn/T/ipykernel_15339/1255237194.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_data = pd.read_csv("title.basics.tsv", sep = "\t")


In [ ]:
imdb_data.head()

## Read in the base Oscar dataset and rename all the awards of interest in into common names and filter for only these award rows

In [ ]:
oscar_data = pd.read_csv("the_oscar_award.csv", encoding = 'latin-1')

best_pic_names = ['OUTSTANDING PICTURE', 'OUTSTANDING PRODUCTION', 'OUTSTANDING MOTION PICTURE', 'BEST MOTION PICTURE', 'BEST PICTURE']

best_actor_names = ['ACTOR', 'ACTOR IN A LEADING ROLE']

best_actress_names = ['ACTRESS', 'ACTRESS IN A LEADING ROLE']

best_director_names = ['DIRECTING', 'DIRECTING (Comedy Picture)', 'DIRECTOR (Dramatic Picture)' ]

best_pic_rows = oscar_data['category'].isin(best_pic_names)

best_actor_rows = oscar_data['category'].isin(best_actor_names)

best_actress_rows = oscar_data['category'].isin(best_actress_names)

best_director_rows = oscar_data['category'].isin(best_director_names)

In [ ]:
oscar_data.loc[best_pic_rows, 'category'] = 'Best Picture'

oscar_data.loc[best_actor_rows, 'category'] = 'Best Actor'

oscar_data.loc[best_actress_rows, 'category'] = 'Best Actress'

oscar_data.loc[best_director_rows, 'category'] = 'Best Director'

#od_tail = oscar_data[(oscar_data['category'] == 'Best Picture') & (oscar_data['winner'] == 'TRUE')].tail(15)

#oscar_data[(oscar_data['category'] == 'Best Picture') & (oscar_data['winner'] == 'TRUE')].tail(15)

In [ ]:
oscar_data[(oscar_data['category'].isin(['Best Picture', 'Best Actor', 'Best Actress', 'Best Director']))]

In [ ]:
oscar_data_filtered = oscar_data[(oscar_data['category'].isin(['Best Picture', 'Best Actor', 'Best Actress', 'Best Director']))].reset_index(drop = True)

## Export to a new csv file and then reimport it to avoid doing previous steps again (this is where the `ids` were added between steps manually)

In [ ]:
#oscar_data_filtered.to_csv('oscar_data1', sep = ',', encoding = 'utf-8')

In [3]:
final_oscar_data = pd.read_csv('oscar_data2.csv', encoding = 'latin-1')

In [4]:
final_oscar_data.head()

,year_film,year_ceremony,ceremony,category,name,film,win,total_noms,total_wins,ids
0,1970,1971,43,Best Picture,"Ross Hunter, Producer",Airport,0,10,1,tt0065377
1,1970,1971,43,Best Actress,Carrie Snodgress,Diary of a Mad Housewife,0,1,0,tt0065636
2,1970,1971,43,Best Director,Federico Fellini,Fellini Satyricon,0,1,0,tt0064940
3,1970,1971,43,Best Actor,Jack Nicholson,Five Easy Pieces,0,4,0,tt0065724
4,1970,1971,43,Best Picture,"Bob Rafelson and Richard Wechsler, Producers",Five Easy Pieces,0,4,0,tt0065724


## Merge the data with the titles csv that gives us titles, runtime, and genres

In [5]:
od_merged = final_oscar_data.merge(imdb_data, how = 'left', on = 'ids')

In [6]:
od_merged.head()

,year_film,year_ceremony,ceremony,category,name,film,win,total_noms,total_wins,ids,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,1970,1971,43,Best Picture,"Ross Hunter, Producer",Airport,0,10,1,tt0065377,movie,Airport,Airport,0,1970,\N,137,"Action,Drama,Thriller"
1,1970,1971,43,Best Actress,Carrie Snodgress,Diary of a Mad Housewife,0,1,0,tt0065636,movie,Diary of a Mad Housewife,Diary of a Mad Housewife,0,1970,\N,95,"Comedy,Drama"
2,1970,1971,43,Best Director,Federico Fellini,Fellini Satyricon,0,1,0,tt0064940,movie,Fellini Satyricon,Fellini - Satyricon,0,1969,\N,129,"Drama,Fantasy"
3,1970,1971,43,Best Actor,Jack Nicholson,Five Easy Pieces,0,4,0,tt0065724,movie,Five Easy Pieces,Five Easy Pieces,0,1970,\N,98,Drama
4,1970,1971,43,Best Picture,"Bob Rafelson and Richard Wechsler, Producers",Five Easy Pieces,0,4,0,tt0065724,movie,Five Easy Pieces,Five Easy Pieces,0,1970,\N,98,Drama


## Created a sample test list of indices before trying to retrieve information on all 700 unique film entries in the data at once

In [53]:
test_list = od_merged.head(30)

In [72]:
od_merged['ids'].unique()[:10]

array(['tt0065377', 'tt0065636', 'tt0064940', 'tt0065724', 'tt0065872',
       'tt0066011', 'tt0066026', 'tt0066206', 'tt0066319', 'tt0065797'],
      dtype=object)

## Made 7 dictionaries of all the budget and box office info since the request timed out if done all at once

In [80]:
budgets1 = {}
box_offices1 = {}


for ID in od_merged['ids'].unique()[:100]:
    movie = ia.get_movie(ID[2:])
    try:
        box_office_data = movie.get('box office', {})
        if 'Cumulative Worldwide Gross' in box_office_data:
            box_offices1[ID] = box_office_data['Cumulative Worldwide Gross']
        elif 'Worldwide Gross' in box_office_data:
            box_offices1[ID] = box_office_data['Worldwide Gross']
        elif 'Gross' in box_office_data:
            box_offices1[ID] = box_office_data['Gross']
        else:
            box_offices1[ID] = (pd.NA)
    except:
        box_offices1[ID] = (None)
    try:
        budgets1[ID] = movie.get('box office')['Budget']
    except:
        budgets1[ID] = (pd.NA)

In [87]:
len(od_merged['ids'].unique())

660

In [82]:
budgets2 = {}
box_offices2 = {}


for ID in od_merged['ids'].unique()[100:200]:
    movie = ia.get_movie(ID[2:])
    try:
        box_office_data = movie.get('box office', {})
        if 'Cumulative Worldwide Gross' in box_office_data:
            box_offices2[ID] = box_office_data['Cumulative Worldwide Gross']
        elif 'Worldwide Gross' in box_office_data:
            box_offices2[ID] = box_office_data['Worldwide Gross']
        elif 'Gross' in box_office_data:
            box_offices2[ID] = box_office_data['Gross']
        else:
            box_offices2[ID] = (pd.NA)
    except:
        box_offices2[ID] = (None)
    try:
        budgets2[ID] = movie.get('box office')['Budget']
    except:
        budgets2[ID] = (pd.NA)

In [85]:
budgets3 = {}
box_offices3 = {}


for ID in od_merged['ids'].unique()[200:300]:
    movie = ia.get_movie(ID[2:])
    try:
        box_office_data = movie.get('box office', {})
        if 'Cumulative Worldwide Gross' in box_office_data:
            box_offices3[ID] = box_office_data['Cumulative Worldwide Gross']
        elif 'Worldwide Gross' in box_office_data:
            box_offices3[ID] = box_office_data['Worldwide Gross']
        elif 'Gross' in box_office_data:
            box_offices3[ID] = box_office_data['Gross']
        else:
            box_offices3[ID] = (pd.NA)
    except:
        box_offices3[ID] = (None)
    try:
        budgets3[ID] = movie.get('box office')['Budget']
    except:
        budgets3[ID] = (pd.NA)

In [86]:
budgets4 = {}
box_offices4 = {}


for ID in od_merged['ids'].unique()[300:400]:
    movie = ia.get_movie(ID[2:])
    try:
        box_office_data = movie.get('box office', {})
        if 'Cumulative Worldwide Gross' in box_office_data:
            box_offices4[ID] = box_office_data['Cumulative Worldwide Gross']
        elif 'Worldwide Gross' in box_office_data:
            box_offices4[ID] = box_office_data['Worldwide Gross']
        elif 'Gross' in box_office_data:
            box_offices4[ID] = box_office_data['Gross']
        else:
            box_offices4[ID] = (pd.NA)
    except:
        box_offices4[ID] = (None)
    try:
        budgets4[ID] = movie.get('box office')['Budget']
    except:
        budgets4[ID] = (pd.NA)

In [88]:
budgets5 = {}
box_offices5 = {}


for ID in od_merged['ids'].unique()[400:500]:
    movie = ia.get_movie(ID[2:])
    try:
        box_office_data = movie.get('box office', {})
        if 'Cumulative Worldwide Gross' in box_office_data:
            box_offices5[ID] = box_office_data['Cumulative Worldwide Gross']
        elif 'Worldwide Gross' in box_office_data:
            box_offices5[ID] = box_office_data['Worldwide Gross']
        elif 'Gross' in box_office_data:
            box_offices5[ID] = box_office_data['Gross']
        else:
            box_offices5[ID] = (pd.NA)
    except:
        box_offices5[ID] = (None)
    try:
        budgets5[ID] = movie.get('box office')['Budget']
    except:
        budgets5[ID] = (pd.NA)

In [89]:
budgets6 = {}
box_offices6 = {}


for ID in od_merged['ids'].unique()[500:600]:
    movie = ia.get_movie(ID[2:])
    try:
        box_office_data = movie.get('box office', {})
        if 'Cumulative Worldwide Gross' in box_office_data:
            box_offices6[ID] = box_office_data['Cumulative Worldwide Gross']
        elif 'Worldwide Gross' in box_office_data:
            box_offices6[ID] = box_office_data['Worldwide Gross']
        elif 'Gross' in box_office_data:
            box_offices6[ID] = box_office_data['Gross']
        else:
            box_offices6[ID] = (pd.NA)
    except:
        box_offices6[ID] = (None)
    try:
        budgets6[ID] = movie.get('box office')['Budget']
    except:
        budgets6[ID] = (pd.NA)

In [90]:
budgets7 = {}
box_offices7 = {}


for ID in od_merged['ids'].unique()[600:660]:
    movie = ia.get_movie(ID[2:])
    try:
        box_office_data = movie.get('box office', {})
        if 'Cumulative Worldwide Gross' in box_office_data:
            box_offices7[ID] = box_office_data['Cumulative Worldwide Gross']
        elif 'Worldwide Gross' in box_office_data:
            box_offices7[ID] = box_office_data['Worldwide Gross']
        elif 'Gross' in box_office_data:
            box_offices7[ID] = box_office_data['Gross']
        else:
            box_offices7[ID] = (pd.NA)
    except:
        box_offices7[ID] = (None)
    try:
        budgets7[ID] = movie.get('box office')['Budget']
    except:
        budgets7[ID] = (pd.NA)

## Turned the 7 dictionaries into dataframes, and then merged them with the original dataset

In [91]:
box_offices = {k: v for d in [box_offices1, box_offices2, box_offices3,
                              box_offices4, box_offices5, box_offices6, box_offices7] for k, v in d.items()}

In [92]:
budgets = {k: v for d in [budgets1, budgets2, budgets3, budgets4,
                          budgets5, budgets6, budgets7] for k, v in d.items()}


In [96]:
box_office_df = pd.DataFrame(box_offices.items(), columns=['ids', 'Box_Office'])

In [97]:
budget_df = pd.DataFrame(budgets.items(), columns=['ids', 'Budget'])

In [99]:
od_merged = od_merged.merge(box_office_df, on='ids', how='left')
od_merged = od_merged.merge(budget_df, on='ids', how='left')

In [102]:
od_merged

,year_film,year_ceremony,ceremony,category,name,film,win,total_noms,total_wins,ids,...,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,Box_Office_x,Box_Office_y,Budget
0,1970,1971,43,Best Picture,"Ross Hunter, Producer",Airport,0,10,1,tt0065377,...,Airport,Airport,0,1970,\N,137,"Action,Drama,Thriller","$371,898","$371,898","$10,000,000 (estimated)"
1,1970,1971,43,Best Actress,Carrie Snodgress,Diary of a Mad Housewife,0,1,0,tt0065636,...,Diary of a Mad Housewife,Diary of a Mad Housewife,0,1970,\N,95,"Comedy,Drama",<NA>,<NA>,<NA>
2,1970,1971,43,Best Director,Federico Fellini,Fellini Satyricon,0,1,0,tt0064940,...,Fellini Satyricon,Fellini - Satyricon,0,1969,\N,129,"Drama,Fantasy",<NA>,<NA>,"$3,000,000 (estimated)"
3,1970,1971,43,Best Actor,Jack Nicholson,Five Easy Pieces,0,4,0,tt0065724,...,Five Easy Pieces,Five Easy Pieces,0,1970,\N,98,Drama,<NA>,<NA>,"$1,600,000 (estimated)"
4,1970,1971,43,Best Picture,"Bob Rafelson and Richard Wechsler, Producers",Five Easy Pieces,0,4,0,tt0065724,...,Five Easy Pieces,Five Easy Pieces,0,1970,\N,98,Drama,<NA>,<NA>,"$1,600,000 (estimated)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161,2024,2025,97,Best Picture,"Coralie Fargeat, Tim Bevan, Eric Fellner",The Substance,0,5,1,tt17526714,...,The Substance,The Substance,0,\N,\N,\N,Drama,<NA>,<NA>,"$17,500,000 (estimated)"
1162,2024,2025,97,Best Actress,Demi Moore,The Substance,0,5,1,tt17526714,...,The Substance,The Substance,0,\N,\N,\N,Drama,<NA>,<NA>,"$17,500,000 (estimated)"
1163,2024,2025,97,Best Director,Coralie Fargeat,The Substance,0,5,1,tt17526714,...,The Substance,The Substance,0,\N,\N,\N,Drama,<NA>,<NA>,"$17,500,000 (estimated)"
1164,2024,2025,97,Best Picture,Marc Platt,Wicked,0,10,2,tt1262426,...,Wicked,Wicked: Part One,0,2024,\N,\N,"Comedy,Drama,Family","$251,313,000","$251,313,000","$150,000,000 (estimated)"


## Finally exported the almost finished dataset (manually checked all NA box_office and budget rows to see if the information was available elsewhere after exporting this file)

In [103]:
od_merged.to_csv('oscar_data3.csv', sep = ',', encoding = 'utf-8')